1 - Get 100 crypto assets with the highest marketcap

2 - Get data

3 - Labeling


    

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels
from statsmodels.tsa.stattools import coint

In [ ]:
# valid intervals - 1min, 3min, 5min, 15min, 30min, 1H, 2H, 4H, 6H, 8H, 12H, 1D, 3D, 1W, 1M
interval = '1H'
# USDT base
#pairs = ['BTCUSDT', 'ETHUSDT','ADAUSDT','CHZUSDT','ENJUSDT','LINKUSDT','FILUSDT','XRPUSDT', 'DOTUSDT','SOLBTC']
# BTC base
#pairs = ['ADABTC','ENJBTC','SOLBTC', 'BNBBTC', 'CHZBTC', 'MANABTC']
#pairs = ['ETHBTC','ADABTC','CHZBTC','ENJBTC','LINKBTC','FILBTC','XRPBTC', 'DOTBTC','SOLBTC', 'BNBBTC']
#recomendations from Fausto Botelho in 29/07/2021 (https://www.youtube.com/channel/UCFzEG7N47dktszRxdfOHJdA)
#pairs = ['BATBTC', 'BNBBTC', 'CHZBTC', 'ETHBTC', 'KNCBTC', 'MATICBTC', 'XEMBTC', 'ONTBTC', 
#    'CAKEBTC', 'RENBTC', 'XRPBTC', 'SOLBTC', 'STEEMBTC', 'SUSHIBTC', 'TRXBTC', 'UNIBTC']
#pairs = ['EGLDBTC', 'FETBTC', 'SKLBTC', 'KAVABTC', 'SXPBTC']
#DEX (Descentralized Exchange)
#pairs = ['UNIBTC', 'SUSHIBTC', 'CAKEBTC', 'DYDXBTC']

#pairs = ['SOLBTC', 'ENJBTC', 'LINKBTC', 'MATICBTC', 'UNIBTC']
pairs = ['ATOMBTC', 'LINKBTC', 'AAVEBTC', 'VETBTC']

#pairs = ['BTCUSDT', 'SHIBUSDT']
# date to start/end analysis
dt_start_analysis = '2021-1-1'
# Use '' for analysis until now
dt_end_analysis = ''

In [ ]:
def normalize_data(df):
    return df/df.iloc[0,:]

In [ ]:
def find_cointegrated_pairs(data):
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    selected_pairs = []
    for i in range(n):
        for j in range(i+1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            result = coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.05:
                selected_pairs.append((keys[i], keys[j]))
    
    return score_matrix, pvalue_matrix, selected_pairs

In [ ]:
from binance_utils import init
from trade_utils import get_data

client = init()

crypto_pairs_dict = {}
crypto_df = pd.DataFrame()

for pair in pairs:
    df = get_data(client, pair, interval)

    if dt_end_analysis == '' and dt_start_analysis != '':
        df = df[dt_start_analysis:]
    elif dt_end_analysis != '' and dt_start_analysis == '':
        df = df[:dt_end_analysis]
    elif dt_end_analysis != '' and dt_start_analysis != '':
        df = df[dt_start_analysis:dt_end_analysis]

    crypto_pairs_dict[pair] = df.copy()

    if crypto_df.empty:
        crypto_df = df[['ClosePrice']].copy()
        crypto_df = crypto_df.rename(columns={'ClosePrice' : pair})
    else: 
        crypto_df = crypto_df.join(df[['ClosePrice']].copy())
        crypto_df = crypto_df.rename(columns={'ClosePrice' : pair})


In [ ]:
crypto_df.head()

In [ ]:
# Example
pair = pairs[0]

crypto_pairs_dict[pair].tail()

In [ ]:
ax = crypto_df.plot(title='Coin Prices')
ax.set_xlabel('Date')
ax.set_ylabel('Price')
plt.show()

In [ ]:
ax = normalize_data(crypto_df).plot(title='Coin Prices')
ax.set_xlabel('Date')
ax.set_ylabel('Normalized Price')
plt.show()

In [ ]:
daily_returns = pd.DataFrame()

for pair in pairs:
    daily_returns_pair = pd.DataFrame(crypto_pairs_dict[pair]['ClosePrice'].resample('1d').ffill().pct_change())
    daily_returns_pair = daily_returns_pair.rename(columns={'ClosePrice' : pair})

    if daily_returns.empty:
        daily_returns = daily_returns_pair.copy()
    else:
        daily_returns = daily_returns.join(daily_returns_pair)

daily_returns.head()

In [ ]:
monthly_returns = pd.DataFrame()

for pair in pairs:
    monthly_returns_pair = pd.DataFrame(crypto_pairs_dict[pair]['ClosePrice'].resample('M').ffill().pct_change())
    monthly_returns_pair = monthly_returns_pair.rename(columns={'ClosePrice' : pair})

    if monthly_returns.empty:
        monthly_returns = monthly_returns_pair.copy()
    else:
        monthly_returns = monthly_returns.join(monthly_returns_pair)

monthly_returns.head()

In [ ]:
#plt.figure(figsize=(4, 3))
daily_returns.plot(title='Daily returns data')
plt.show()

In [ ]:
monthly_returns.plot(title='Monthly returns data')
plt.show()

In [ ]:
daily_returns.hist(bins = 60)
plt.show()

In [ ]:
# compute daily returns in the same plot
for pair in pairs:
    daily_returns[pair].hist(bins=60, label=pair)

plt.legend(loc='upper right')
plt.show()

In [ ]:
cum_returns = (daily_returns + 1).cumprod()

In [ ]:
ax1 = (daily_returns + 1).cumprod().plot()
ax1.set_xlabel("Date")
ax1.set_ylabel("Growth of $1 investment")
ax1.set_title("Daily cumulative returns data")
plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_axes([0.1,0.1,0.8,0.8])
cum_returns = (monthly_returns + 1).cumprod()
ax1.plot(cum_returns)
ax1.set_xlabel("Date")
ax1.set_ylabel("Growth of $1 investment")
ax1.set_title(pair + " Monthly cumulative returns data")
plt.show()

In [ ]:
ax1 = (monthly_returns + 1).cumprod().plot()
ax1.set_xlabel("Date")
ax1.set_ylabel("Growth of $1 investment")
ax1.set_title("Monthly cumulative returns data")
plt.show()

In [ ]:
print(monthly_returns.mean())

In [ ]:
print(monthly_returns.std())

In [ ]:
print(monthly_returns.corr())

In [ ]:
print(monthly_returns.cov())

In [ ]:
import seaborn

# Heatmap to show the p-values of the cointegration test
# between each pair of stocks
scores, pvalues, selected_pairs = find_cointegrated_pairs(crypto_df)
m = [0,0.2,0.4,0.6,0.8,1]
seaborn.heatmap(pvalues, xticklabels=pairs, 
                yticklabels=pairs, cmap='RdYlGn_r' 
                , mask = (pvalues >= 0.98)
                )
plt.show()
print(selected_pairs)

In [ ]:
S1 = crypto_df['CHZBTC']
S2 = crypto_df['MANABTC']
score, pvalue, _ = coint(S1, S2)
print(pvalue)
ratios = S1 / S2
ratios.plot(figsize=(15,7))
plt.axhline(ratios.mean())
plt.legend(['Price Ratio'])
plt.show()

In [ ]:
def zscore(series):
    return (series - series.mean()) / np.std(series)

In [ ]:
zscore(ratios).plot(figsize=(15,7))
plt.axhline(zscore(ratios).mean(), color='black')
plt.axhline(1.0, color='red', linestyle='--')
plt.axhline(-1.0, color='green', linestyle='--')
plt.legend(['Ratio z-score', 'Mean', '+1', '-1'])
plt.show()